#                     Time Series Analysis on the
#        Monthly Retail Trade Servey (MRTS) Data

**Chris Machado**



# Index

<img align="right" width="400" src="ecommerce.jpg">

- [Abstract](#Abstract)
- [1. Introduction](#1.-Introduction)
- [2. Extract-Transform-Load](#2.-Extract-Transform-Load)
    - [2.1 The ETL Process](#2.1-The-ETL-Process)
    - [2.2 Data Exploration](#2.2-Data-Exploration)
    - [2.3 Data Preparation](#2.3-Data-Preparation)
    - [2.4 Read the Data Using Python](#2.4-Reading-the-Data-Using-Python)
         - [2.4.1 Reading Sample Data](#2.4.1-Reading-Sample-Data)
         - [2.4.2 Reading the MRST Data](#2.4.2-Reading-the-MRST-Data)
    - [2.5 Writing an Installation Script](#2.5-Writing-an-Installation-Script)
- [3. Analysis and Visualization](#3.-Analysis-and-Visualization)
    - [3.1 Running Queries in MySQL Workbech](#3.1-Running-Queries-in-MySQL-Workbech)
    - [3.2 Running Queries From Python](#3.2-Running-Queries-From-Python)
    - [3.3 Explore Trends](#3.3-Explore-Trends)
    - [3.4 Explore Percentage Change](#3.4-Explore-Percentage-Change)
    - [3.5 Explore Rolling Time Windows](#3.5-Explore-Rolling-Time-Windows)
- [Conclusion](#Conclusion)
- [References](#References)

[Back to top](#Index)


##  Abstract

The Monthly Retail Trade Survey (MRTS) dataset contains estimated spending activity across the U.S. going back 28 years and for 59 different business categories. The Extract-Transform-Load (ETL) sequence is used to upload onto a MySQL server. From there, data is queried using python, and multiple time series are analyzed

The plots reveal details about business sectors and consumer spending habits. Furniture stores are susceptible to recessions while e-commerce isn't. Internet spending is growing in popularity, while books stores are on the decline.

[Back to top](#Index)


## 1. Introduction

In this project, I journey through the process of taking raw data from the public domain, and transforming it into relevant information. 

Starting with an Excel file, I work through the Extract-Transform-Load (ETL) process. I reshape and clean it using python. Then develop a loading script where the data is stored on a local MySQL server.

Using MySQL Workbench as test bench for writing queries, I move them over, embedding into a python script. This targeted data is then used for analysis. I look at several time series that show increasing and decreasing trends, impacts felt from recessions, and even holiday spending patterns. I make use of percentage change, and explore rolling time windows to help make the charts more readable.

[Back to top](#Index)

## 2. Extract-Transform-Load


#### The full ETL script can be found [here](Project8.1-ETL-Machado.py).

[Back to top](#Index)

### 2.1 The ETL Process

Prior to analyzing the dataset, it needs reformatting and placement into a repository for future and easier access. This was accomplished through the extract, transform, and load process. The dataset is available for public download at the [census.gov](https://www.census.gov/retail/mrts/historic_releases.html) website in an Excel document, going back to 1992. 

The data is organized by year in separate worksheets, then further divided by month with 60 business categories. Extraction was completed by looping over each of the worksheets pulling the relevant data. It was then transformed by reshaping and cleaning the data, exporting a tidy csv file to the local directory. 

Finally the transformed data was loaded from an integrated development environment (IDE) to a MySQL server by use of a driver and script looping over each of the rows.

[Back to top](#Index)

### 2.2 Data Exploration

The MRTS data is an estimate of total U.S. retail sales by category, collected each month. The surveys capture current economic activity in the retail trade and food service sectors. The data is derived from a sample drawn from the businesses register containing all U.S. retail businesses with paid employees. The sample size is approximately 13,000.

Many economic indicators are derived from the MRTS. Gross Domestic Product (GDP), Consumer Price index (CPI), and Producer Price Index (PPI) are just a few. These indicators help gauge the health of the U.S. economy and guide policy decisions. 

Businesses also leverage the data. Through analysis, market trends and consumer habits are used to make decisions in the best interest of the organization. 

[Back to top](#Index)

### 2.3 Data Preparation

<img style="float: right;" src="excel1.png"/>

</br>

* The data we are interested in is embedded in the Excel file surrounded by a few unwanted cells and columns. The title and header rows don't contain data and need to go. 


* The bottom half contains seasonally adjusted data. Because it is modified, we will exclude for analysis. 


* The first several rows and last column contain totals. Those can be calculated later if needed. 


* The North American Industry Classification System (NAICS) codes also won’t help in the analysis.  


[Back to top](#Index)

### 2.4 Read the Data Using Python

Python can ingest the data in a couple different ways. The first is through the comma separated variables (csv) module. The Excel document needs to first be exported as a csv file. Python can then open and read through the csv.reader function. This method allows for quick and easy shaping functions in excel (i.e. deleting columns and rows), however, each spreadsheet needs to be altered and exported separately.

Another method is through use of pandas read_excel function. When executed, the spreadsheet is loaded as a dataframe. A looping script can quickly ingest the data from each sheet.

There are several other libraries out there designed to work with .xls/.xlsx files. I won't go over them, since I chose to use the pandas method.

[Back to top](#Index)

### 2.4.1 Reading Sample Data

Before trying to ingest a large dataset, it is a good idea to start small and ensure the scripts function as intended. I created a simple file, test.xls, then loaded it with: 

In [2]:
import pandas as pd

test_data = pd.read_excel('test.xls') 
test_data

,Item,color,shape,weight
0,Apple,red,sphere,1.5
1,Orange,orange,sphere,2.2
2,Pear,green,oval,1.7
3,Bananna,yello,oblong,2.3
4,Watermellon,green,oval,24.0


[Back to top](#Index)

### 2.4.2 Reading the MRTS Data

#### Extract and Transform
<img style="float: left;" src="script-cutline.png"/>

Extracting the data is accomplished by looping over each year's spreadsheet, 1992 to 2020, and loading the desired cells into a dataframe. Each year's data is then reshaped and cleaned before being concatenated to a single dataframe *prep_data*.  

Fortunately the spreadsheet layout has remained fairly consistent over the years. Business categories did not change, and all data remains in the same cell numbers, making it easier to extract. The only observed changes to the structure are in the notes section at the bottom.

Prior to loading the file, it is important to identify where the footer begins. In other words, what bottom rows should be excluded. The read_csv function has both skiprows and skipfooter options that partitions the top and bottom user defined rows respectively. Noticing the footer changed in 2001 and 2017, I declared a variable *cut_line* that specifies how many rows to include in the footer, depending on the year.    

<img style="float: left;" src="script-loadfile.png"/>

The dataframe *data* is defined, extracting from the desired worksheet, and omitting the header, unwanted rows, and footer.

Next, I dropped the columns *TOTAL* and *NAICS Code*. Then assigned the name *dates* to the empty top left cell. 

</br></br>

The presence of apostrophes in the business categories column will cause errors later when loaded into database queries, so they were removed. 



<img style="float: left;" src="script-transpose.png"/>

Logically structuring the dataframe for use in a database requires swapping the column headers and indexes. For this, the transpose function is called ensuring our rows contain sequential dates and our columns renamed to correspond with the type of business in a new dataframe *data_T*.

<img style="float: left;" src="script-date.png"/>

The dates are in a non-standard format (*MMM. YYYY*). Since the intended destination is an SQL database, a conversion is needed to a format accepted by the server. This was done by first splitting string on the period ".," so month and year became separate items. 

At this point I discovered the month of may did not have a period. Under further inspection, I found this to be true for each year. It seems as though this minor detail was in the Census Borough’s original scripting, and the error carried forward since 1992. To fix, I needed to manually enter the period in the original Excel document.

The *dates* column was dropped after extracting both the month and year, now in the form of tuples, into their own respective series. Instead of spelling out the month, I used the replace function to convert each into a two-digit string. Then the string was reassembled into a standard SQL date format (YYYY-MM-DD). Because the temporal resolution is only monthly, I arbitrarily chose the first day of the month.
The new list is inserted into *data_T* as a new column in the left most position titled *Date*.  

Some of the data was missing as indicated by (NA) for "Not Available" and (S) for "Sampling Error." The replace function was called again, placing the value 0 instead.  

<img style="float: left;" src="script-concat.png"/>

</br>
With the data reshaped and cleaned, it is concatenated with the *prep_data* dataframe before the loop recycles and loads another year's spreadsheet.

Finally, with each sheet iterated trough forming a single dataframe, the index needs resetting because each row carried with it the indices from *data_T* dataframe.

<img style="float: left;" src="script-resetind.png"/>

To ensure all that work is preserved, *to_csv()* is invoked, saving a csv file to the present directory. 

#### The full ETL script can be found [here](Project8.1-ETL-Machado.py).

[Back to top](#Index)

### 2.5 Writing an Installation Script

#### Build the database

Now that the data is nice and neat, it is time to load into the MySQL server. To do this, I first need to build a database with table. I could make use of MySQL Workbench, but I will need to define 60 columns for my table. It is much easier to write a script and automate the entire process using the mysql-connector driver.

<img style="float: left;" src="script-sqlconnect.png"/>

First, I should define how I will connect to the server. My server and login credentials are stored in a .yaml configuration file located in my parent directory. With the assistance of the yaml library I am able to import the file and define the configuration.

The driver establishes a connection using the connect function, passing the configuration. By default, any query that alters data will not take effect unless an explicit *commit()* statement is called. This behavior is changed through by enabling the autocommit feature.  

A cursor object is also defined. It will be used to make SQL statements. 

Before creating the database, several things need to be decided first. Since there is only one dataframe with unique values, a single table will suffice. The table's column headers will correspond to the dataframes. Since the date should be unique for each row, it could be used as the primary key. However, I decided to add an additional column, *id*, to act as the primary key. To assist with the task *cols* is defined, extracting the dataframe's column names.

<img style="float: left;" src="script-createtable.png"/>
A query statement is assembled in three sections to build the database infrastructure. The first part consists of three separate commands separated by semicolons and the beginning of a fourth command. 

In the event the server already contains a database named *mrts*, we first delete it by dropping it. If it doesn't exist, the added statement *IF EXISTS* is there to ensure an error doesn't interrupt execution. After creation, it is selected through the *USE* statement. Any new commands will be directed at the intended database.

<br>The fourth command begins at *CREATE TABLE*, with *id* and *Date* explicitly defined as the first two columns. The MRTS data contains integers representing U.S. dollar amounts in millions. *Date* is the only exception, and the reason it is treated separately as a *DATETIME* object.

The second section of the query statement begins with a *for* loop. It iterates through the column names skipping *Date*, and appending each to the statement as integer types. 

Following the loop, the query is closed out, defining which column serves as the primary key, the database engine, and the character set used. The commands are sent to the server via the cursor's execute function with the *multi* option enabled since the statement contains four separate commands.

<img style="float: left;" src="script-sqldisconnect.png"/>

As a good practice, close the cursor and connection to the SQL server.

#### Populate the database

With a database and table built, it can be populated with data.

<img style="float: left;" src="script-sqlconnect2.png"/>  

Establish the connection and cursor as before. Don't forget to enable autocommit.

The dataset contains 348 rows and 59 columns. Automated entry into the database requires a loop to iterate through each row of the dataframe, construct an *INSERT* query, then execute. As this is performed, *i* is denoted as the row's index, and *r* the values as a series.

<img style="float: left;" src="script-sqlquery2.png"/>
Because the field *id* is designed to auto increment, it doesn't need inclusion. However, omitting it creates a mismatch between the number of table columns and number of data points. This scenario requires each value to be mapped to the intended column, thus resolving any ambiguity. But because there are 59 values in each row, it's much cleaner to just include it by referencing the rows index *i*. 

<br>Since the index begins at *0*, it is incremented by *1* before joining the row as a list *rv*. The *INSERT* query is constructed as a string containing sixty string variables denoted by *%s*. The cursor executes the statement by passing both the string and list of values corresponding to each of the variables.

<img style="float: left;" src="script-sqldisconnect.png"/> 

Again, close the cursor and disconnect. The table is populated.

#### The full ETL script can be found [here](Project8.1-ETL-Machado.py). 


[Back to top](#Index)

## 3. Analysis and Visualization

#### The entire script used for the analysis can be found [here](Project8.2-TS-Analysis-Machado.py). 

[Back to top](#Index)

### 3.1 Running Queries in MySQL Workbech

With the dataset now loaded onto the server, queries can be run to access it. Tools available in Python are ideal for data manipulation and plotting. But before jumping into a scrip, I ran several queries on MySQL Workbench first. In doing so, I verified data was uploaded correctly, and ensured my statements would work as intended before inserting them into a script. 

<img align="right" src="sql_query.png"> 
<br>Verifying the data loaded corectly involved running a *sum* query for a business type at the begining and end of the list, and only for years 1992 and 2020  in this case, "Motor vehicle and parts dealers" and "Limited service eating places.:

<br>```SELECT sum(`Motor vehicle and parts dealers`) 
AS 'Total'
FROM mrts_data
WHERE year(`Date`) = 1992```

<br>In each case, the number returned matched the value on the original Excel file, providing confidence the data was loaded correctly.

I performed additional queries on MySQL Workbench during the analysis, but before placing into my script. The image to the right is an example used to find the total money spent on electronic purchases grouped by month. 

[Back to top](#Index)

### 3.2 Running Queries From Python

To keep the script relatively tidy, I defined a few functions. One of which performs all the required actions to execute a query statement, and return a pandas DataFrame.


<img align="left" src="func_sql_query.png"> 

The user provides the statement as a string and the configuration file. The auto_commit feature is defaulted to off, but provides the option to enable it. The process to connect and disconnect is the same as described above, but a loop is now required to extract each line of data from the cursor's *fetchall* function.  

The following queries were made from the IDE during and used for the analysis:

```
query = """
SELECT `id`, `Date`, `Furniture stores` FROM mrts_data
"""
query = """
SELECT `id`, `Date`, `Electronic shopping and mail-order houses` FROM mrts_data
"""
```
<br><br>
```
query = """
SELECT month(`Date`) AS 'Month', 
sum(`Electronic shopping and mail-order houses`) AS 'Monthly Totals' 
FROM mrts_data 
GROUP BY month(`Date`);
"""
query = """
SELECT `id`, `Date`, `Book stores` FROM mrts_data
"""
```

Each of these statements were then called via the *sql_query* function

```
eShoping = sql_query(query, '../mrts.yaml') 
```

[Back to top](#Index)

### 3.3 Explore Trends


Trend analysis is helpful to understand the environment and make predictions about the future. Given the MRTS dataset, we can derive macroeconomic details through the time series. 

Although before making any charts, it is important to understand inflations role on spending power over time. A dollar's value is generally worth less as time progresses, and is a characteristic of a healthy economy. Regulators try to keep inflation in check with an annual rate of around 2%. With that in mind, if the pattern of spending behavior didn't change over years, we still expect to see an upward trend in spending due to inflation. 

Take for example the image below, showing the money spent at furniture stores over time. The left plots raw unaltered data, while the right adjusts the values to today's dollar value. The differences are most prominent at the beginning and end. Each show upward trends, meaning Americans are spending more on furniture over time. But the rate of change is significantly lower when adjusted.

<img align="left" width="730" src="furniture.png"> 

There are a few other takeaways from the above figure. Downward trends are observed just after the years 2000 and 2007. Those time periods coincide with the dot-com bust and great recession. When compared to other business categories, such as book stores (below), furniture are particularly sensitive to economic downturns. When adjusted for inflation, those downward trends are amplified.

Following the great recession, Americans spending in furniture stores slowly picked up. However, spending levels haven't yet reached their pre-recession peak. A detail not captured in the unadjusted graphic.  

The sharp downward spike around 2020, is a grim reminder of the Corona virus lockdown, where many businesses were forced to close.  

<img align="left" src="func_inflate.png"> 
<img align="left" src="func_make_plot.png"> 

Correcting for inflation is accomplished with the displayed function inflate. It accepts a dataframe needing adjustment and another with inflation figures. 

The [World Bank](https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG?locations=US) tracks inflation numbers for nearly every country on their website. The U.S. data from 1992 - present, is transferred into a new [spreadsheet](usinflation.xls) for ease of reading. 

A for loop multiplies each value by itself plus the inflation rate, for each year to the present. 

For ease of plotting the make_plot function is defined to the left. I found the need to rotate the x axis labels for a few plots, so I and included it, color, and save file as options. 

<img align="right" src="eshoppingtrend.png"> 

*Electronic shopping and mail-order houses* is a business category from the MRTS set that stood out since it captures all purchases made over the internet. Because records begin in 1992, it is fascinating to see trend evolve as the internet made its debut. 

What stands out is a logarithmic trend accelerating with each successive year. The impact of the two previous recessions is barely noticeable. The 2020 lockdown also had no observable affect dampening spending as expected. Some argue that online shopping received a boost from the corona virus. While there is a sharp increase in spending during 2020, the rise doesn't deviate from the logarithmic pattern. Future data is needed to draw any conclusions. 
  


<img align="right" src="eshoppingmonthly.png"> 

Another obvious feature, is the presence of spikes appearing each year. I suspect these anomalies occur during the Christmas holiday season, when holiday shoppers begin feeling generous. 

<br>To confirm this hypothesis, I summed total spending from 1992 to 2020 and grouped by month. The results are presented in the bar graph to the right. It looks as though November and December spending habits are nearly \$300 million above normal.

[Back to top](#Index)

### 3.4 Explore Percentage Change

<img align="right" src="eshoppingpercent.png"> 

Expanding on the trends in internet shopping I was curious to understand just how much of the economy depends on it. 

The plot to the right represents internet spending as a proportion to the whole. In other words, it is a percentage of the total spending across all categories. The plot looks very similar to the one above, but there are some key differences. 

The trend change for the recession beginning in 2000 is more pronounced, while the 2008 recession is unobservable. As a percentage of spending, a decrease represents a disproportionate pullback when compared to all other business categories.

As expected the overall trend is upward, meaning each year people will spend more on the internet than will in other places. Although, I was disappointed in the numbers and expected to see more. Presently, only 5% to 6% of sales occur online. That leaves a lot of room for growth. 

In 2020, there is a noticeable spike not seen in the previous graphic. It is unclear how much of the spike is due to increased spending from those trapped in their homes, or a result of not being able to spend in every other category.


[Back to top](#Index)

### 3.5 Explore Rolling Time Windows


<img align="right" src="bookstrend1.png">
<img align="right" src="bookstrendrolling.png"> 

<br><br>The rolling time window is a method to smooth data with large variation. Take for instance bookstore sales from the MRTS dataset. The overall trend is obvious, book store sales are declining and on there way out. However, in a shorter time span it is difficult to discern what is happening. There appear to be wide fluctuations in amplitude that don't correlate to the holiday spending pattern.    

<br>To smooth the data, I took the average of a three month period and recorded it as the value for the center month. Then increment by one month and repeat. For example, values for (JAN, FEB, MAR) were (1, 1, 4) respectively. The average is 2, and that value is recorded for FEB. Then do the same for (FEB, MAR, APR) repeating through the entire data set. 

<br>The result of this method is presented to the right. Notice how the peaks and troughs are dampened, yet the overall trend is preserved. With the introduction of e-readers, smartphones, and tablets, around 2010, it is no surprise to see sales slowly decline year after year. 

<img align="right" src="bookszoom1.png">
<img align="right" src="bookszoom2.png"> 

<br>Zooming into the reshaped time series, shows other interesting features. The graph to the right shows a period of 24 months spanning 2018 and 2019. That time was selected to represent the most current book store spending habits, without the effects induced by corona virus response efforts. 

<br>Just below, is a similar plot show a period spanning 2006 and 2007. The time frame is just prior to the great recession and marks the point where sales begin to fall.

<br>In both plots, a pattern emerges. Two peaks are seen in each year, one in late summer to early autumn, the other in winter coinciding with the holiday shopping season. This first sudden increase in spending could be the result schools reopening after summer break. Or, it could be attributed to increased summer travel, where reading material helps pass the time on a long trip.

<br>The bottom plot, shows spending habits twelve years earlier, and is also telling. The winter peak is substantially higher, \$200 million higher. That trend is observed in nearly every year when examining the adjusted time series. However, in recent years the difference has significantly decrease to the point where both peaks are approximately equal. Not only are sales declining in general, it appears as though people are also spending less on books for gifts. 


#### The entire script used for the analysis can be found [here](Project8.2-TS-Analysis-Machado.py). 

[Back to top](#Index)

## Conclusion

The MRTS dataset contains a trove of information to be used for market analysis. Through the ETL process I was able to reshape, clean and load the data to an SQL server. Then using python, pull targeted columns for analysis. 

Plotting time series, I showed how vulnerable furniture stores are to recessions, and how ecommerce isn't. The charts also revealed how e-shopping is gaining popularity and bookstores are losing it.

There are numerous more analyses that could be performed the data, and the ones covered here are  only a small sample of the possibilities.


[Back to top](#Index)
## References


"E-Commerce In Our Daily Life." Dash Technologies. Domains By Proxy, 2021. https://dashtechinc.com/e-commerce-in-our-daily-life/

"Inflation, Consumer Prices (annual%) Data." World Bank Data. World Bank Group, 2022. https://data.worldbank.org/indicator/FP.CPI.TOTL.ZG

McKinney, Wess and Kluyver, Thomas. “pandas: Powerful Python Data Analysis Toolkit - Version 1.4.3." The Pandas Development Team, 2022. https://pandas.pydata.org/docs/pandas.pdf

"Monthly Retail Trade." United States Census Bureau. U.S. Department of Commerce, 2022. https://www.census.gov/retail/index.html